In [ ]:
from music21 import converter , instrument , chord ,note , stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils_utils

In [ ]:
midi = converter.sparse( 'location.mid')

In [ ]:
midi

In [ ]:
midi.show('midi')

In [ ]:
midi.show('text')

In [ ]:
element_to_parse = midi.flat.notes

In [ ]:
len(element_to_parse)

In [ ]:
for e in element_to_parse:
    print(e,e.offset)

In [ ]:
notes_demo = []

for ele in element_to_parse:
    if isinstance(ele,notes.Notes):
        notes_demo.append(str(ele.pitch))
    
    elif isinstance(ele,chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normaloOrder))
        
    
    
    
     

In [ ]:
notes_demo

# Processing all files

In [ ]:
notes = []


for files in glob.glob('/midisongs/.midi'):
    midi = converter.parse(file)
    
    print('parsing is%'%file)
    
    element_to_parse = midi.flat.notes
    
    
    for ele in element_to_parse:
        if isinstance(ele,notes.Notes):
            notes_demo.append(str(ele.pitch))

        elif isinstance(ele,chord.Chord):
             notes_demo.append("+".join(str(n) for n in ele.normaloOrder))

   
    
    
    

In [ ]:
notes
len(notes)

In [ ]:
#save notes 
with open('notes','wb') as filepath:
    pickle.dump(notes,filepath)

In [ ]:
with open('notes','rb') as f:
    notes = pickle.load(f)

In [ ]:
n_vocab = len(set(notes))

In [ ]:
print("total notes - ",len(notes))
print("unique notes - ",n_vocab)

In [ ]:
print(notes[:50])

# Prepare sequential data for LSTM

In [ ]:
sequence_length = 100

In [ ]:
pitch_name = sorted(set(notes))

In [ ]:
ele_to_int = dict((ele,num) for ele ,num in enumerate(pitch_name)

In [ ]:
network_input  = []
network_output = []

In [ ]:
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i + sequence_length]
    seq_out = notes[i + sequence_length]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [ ]:
network_input

In [ ]:
n_pattern  = len(network_input)
print(n_pattern)

In [ ]:
network_input = np.reshape(network_input,(n_pattern,sequence_length,1))
print(network_input.shape)

In [ ]:
network_input

In [ ]:
normalized_network__input = network_input/(n_vocab)

In [ ]:
network_output = np.utils.to_categorical(network_output)

In [ ]:
print(normalized_network_input.shape)
print(network_output.shape)

# Create Model

In [ ]:
from keras.model import sequential , load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint , EarlyStopping

In [ ]:
model = sequential()
model.add(LSTM(units = 512,
               input_shape = (normalized_network_input.shape[1],normalized_network_input[2]),
               return_sequences = True)
model.add(Dropout(0.3))
model.add(LSTM(units = 512 , return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(units = 512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab),activation = 'softmax')

model.compile(loss = 'categorical_cross_entropy',optimizer = "adam")
          
          
          

In [ ]:
model.summary()

In [ ]:
checkpoint = model.ModelCheckpoint('model.hdf5',monitor = 'loss' , verbos = True ,save_best_only = True , mode = 'min')
model_his = model.fit(normalized_network_input,network_output,epochs = 100 , batch_size = 64 , callbacks = [checkpoint ])

In [ ]:
model = load_model('hdf5')

# Predictions

In [ ]:
sequence_length = 100
network_input = []

for i in range (len(notes) - sequence_length):
    seq_in = notes(i:i + sequence_length )
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [ ]:
start = np.random.randint(len(network_input) - 1 )

In [ ]:
#mapping int to ele
int_to_ele = dict((num ,ele) for num , ele in enumerate(pitch_name))

#initial_pattern
pattern = network_output[start]
prediction_output = []


for note_index in range(200):
    prediction_input = np.reshape(pattern , (1,len(pattern),1))
    prediction_input = prediction_input/(n_vocab)
    
    predict = predict.model(predict_input , verbos = True)
    
    idx = np.argmax(predict)
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]
    

In [ ]:
prediction_output

# Create MidiFile

In [ ]:
offset = 0
output_note = []

for pattern in prediction_output:
    
    if(' + ' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        
    for current_notes in notes_in_chords:
        new_note = notes.Notes(int(current_note))
        new_notes.storedInstrument = instrument.Piano()
        temp_notes.append(new_notes)
        
    new_chord = chord.Chord(temp_notes)
    new_chord.offset = offset 
    output_notes.append(new_chord)
    
    
    else:
        new_notes = notes.Notes(prediction_output[4])
        new_notes.offset = offset
        new_notes.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
        
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = 'test_output.mid')

In [ ]:
midi_stream.show('midi')